In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Model(nn.Module):
  # input layer (7 features) -> hidden layer1 (number of neurons) -> hidden layer2(number of neurons) -> output layer
  def __init__(self, in_features=7, h1=16, h2=16, out_features=3):
    super().__init__()
    self.fc1 = nn.Linear(in_features, h1)
    self.fc2 = nn.Linear(h1, h2)
    self.out = nn.Linear(h2, out_features)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)
    return x

In [3]:
torch.manual_seed(30)
model = Model()

In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("Crop_recommendation.csv")
print(df.head())

    N   P   K  temperature   humidity        ph    rainfall label
0  90  42  43    20.879744  82.002744  6.502985  202.935536  rice
1  85  58  41    21.770462  80.319644  7.038096  226.655537  rice
2  60  55  44    23.004459  82.320763  7.840207  263.964248  rice
3  74  35  40    26.491096  80.158363  6.980401  242.864034  rice
4  78  42  42    20.130175  81.604873  7.628473  262.717340  rice


In [6]:
X = df.drop('label', axis=1).values
y = df['label'].values

In [7]:
print(X.shape, y.shape)
print(X[:5], y[:5])


(2200, 7) (2200,)
[[ 90.          42.          43.          20.87974371  82.00274423
    6.50298529 202.9355362 ]
 [ 85.          58.          41.          21.77046169  80.31964408
    7.03809636 226.6555374 ]
 [ 60.          55.          44.          23.00445915  82.3207629
    7.84020714 263.9642476 ]
 [ 74.          35.          40.          26.49109635  80.15836264
    6.9804009  242.8640342 ]
 [ 78.          42.          42.          20.13017482  81.60487287
    7.62847289 262.7173405 ]] ['rice' 'rice' 'rice' 'rice' 'rice']


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [9]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train) # encodes labels as integers
y_test = label_encoder.transform(y_test)


num_classes = len(label_encoder.classes_)
model = Model(in_features=7, out_features=num_classes)

In [10]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


torch.Size([1760, 7]) torch.Size([440, 7]) torch.Size([1760]) torch.Size([440])


In [11]:
# define the loss function
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0010)

In [12]:
epochs = 200
losses = []

for i in range(epochs):
  y_pred = model.forward(X_train)
  loss = criterion(y_pred, y_train)
  losses.append(loss.item())

  if i % 10 == 0:
    print(f'Epoch: {i}, loss: {loss}')

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

Epoch: 0, loss: 13.660161972045898
Epoch: 10, loss: 8.800141334533691
Epoch: 20, loss: 6.182066917419434
Epoch: 30, loss: 4.558180332183838
Epoch: 40, loss: 3.5597167015075684
Epoch: 50, loss: 2.9506349563598633
Epoch: 60, loss: 2.5311529636383057
Epoch: 70, loss: 2.2705092430114746
Epoch: 80, loss: 2.091113567352295
Epoch: 90, loss: 1.9377676248550415
Epoch: 100, loss: 1.8002116680145264
Epoch: 110, loss: 1.678932785987854
Epoch: 120, loss: 1.572609543800354
Epoch: 130, loss: 1.4748778343200684
Epoch: 140, loss: 1.3850326538085938
Epoch: 150, loss: 1.3006936311721802
Epoch: 160, loss: 1.2202188968658447
Epoch: 170, loss: 1.1435844898223877
Epoch: 180, loss: 1.0712147951126099
Epoch: 190, loss: 1.0032014846801758


In [13]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(range(epochs), losses)
plt.ylabel('Loss')
plt.xlabel('Epoch')

Text(0.5, 0, 'Epoch')

In [14]:
with torch.no_grad():
  y_eval = model.forward(X_test)
  loss = criterion(y_eval, y_test)
loss

tensor(0.9241)

In [15]:
correct = 0
with torch.no_grad():
  for i, data in enumerate(X_test):
    y_val = model.forward(data)

    # print(f'{i+1:2}. {str(data.numpy()):25} {y_val.argmax().item():<10}')

    if y_val.argmax().item() == y_test[i]:
      correct += 1
print(f'{correct} out of {len(y_test)} = {100*correct/len(y_test):.2f}% correct\nNumber of incorrects = {len(y_test) - correct}')

353 out of 440 = 80.23% correct
Number of incorrects = 87


In [16]:
# [90, 42, 43, 20.87974371, 82.00274423, 6.502985292000001, 202.9355362] -> from dataset
# [75, 65, 39, 31.79123169, 50.32964408, 6.098096361, 124.6555374] -> from our own(new data)

new_data = torch.tensor([90, 42, 43, 20.87974371, 82.00274423, 6.502985292000001, 202.9355362])
with torch.no_grad():
  new_data_prediction = model(new_data)
  new_data_index = new_data_prediction.argmax().item()

  predicted_label = label_encoder.inverse_transform([new_data_index])[0]

  print(f'Predicted class index: {new_data_index}')
  print(f'Predicted class label: {predicted_label}')

Predicted class index: 20
Predicted class label: rice


In [17]:
torch.save(model.state_dict(), 'crop_prediction_model.pt')

In [18]:
num_classes = len(label_encoder.classes_)
new_model = Model(in_features=7, h1=16, h2=16, out_features=num_classes)

In [19]:
new_model.eval()

Model(
  (fc1): Linear(in_features=7, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (out): Linear(in_features=16, out_features=22, bias=True)
)

In [20]:
# testing the new_model by providing the new data!!
new_data = torch.tensor([90, 42, 43, 20.87974371, 82.00274423, 6.502985292000001, 202.9355362])
with torch.no_grad():
  new_data_prediction = new_model(new_data)
  new_data_index = new_data_prediction.argmax().item()

  predicted_label = label_encoder.inverse_transform([new_data_index])[0]

  print(f'Predicted class index: {new_data_index}')
  print(f'Predicted class label: {predicted_label}')

Predicted class index: 8
Predicted class label: jute


# Exporting the model so that it could be integrated in android project

In [22]:
# Save the trained model to a .pth file
torch.save(model.state_dict(), "crop_prediction_model.pt")


In [44]:
import joblib

# Assuming label_encoder is used in training
joblib.dump(label_encoder, "label_encoder.pkl")


['label_encoder.pkl']